# Assignment 03 - Due Friday, November 18 at 4pm

*Objectives*: Use Spark to process and perform basic analysis on non-relational data, including its DataFrame and SQL interfaces.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with Python code, SQL queries, shell commands, and markdown cells.  The notebook itself should be completely reproducible (using AWS EC2 instance based on the provided AMI) from start to finish; another person should be able to use the code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking when appropriate.


*Deadline*: Friday, November 18, 4 pm.  

## Part 1 - Setup

Begin by setting up Spark and fetching the project data.  

**Note**: you may want to use a larger EC2 instance type than normal.  This project was prepared using a `t2.xlarge` instance.  Just remember that the larger the instance, the higher the per-hour charge, so be sure to remember to shut your instance down when you're done, as always.

### About the data

We will use JSON data from Twitter; we saw an example of this in class.  It should parse cleanly, allowing you to focus on analysis.

This data was gathered using GWU Libraries' [Social Feed Manager](http://sfm.library.gwu.edu/) application during a recent game of the MLB World Series featuring the Los Angeles Dodgers and Houston Astros.  This first file tells you a little bit about how it was gathered:

#### First make sure you are working from the right working directory

In [1]:
!pwd

/home/ubuntu/notebooks


#### This below file provides context and detail information on the files we are going to work with
#### in this assignment

In [2]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611-README.txt

--2022-11-18 06:33:01--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611-README.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.204.64, 54.231.201.16, 54.231.234.112, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.204.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1920 (1.9K) [text/plain]
Saving to: ‘9670f3399f774789b7c3e18975d25611-README.txt.1’

9670f3399f774789b7c 100%[===================>]   1.88K  --.-KB/s    in 0s      

2022-11-18 06:33:01 (20.3 MB/s) - ‘9670f3399f774789b7c3e18975d25611-README.txt.1’ saved [1920/1920]



In [3]:
!cat 9670f3399f774789b7c3e18975d25611-README.txt

This is an export created with Social Feed Manager.

EXPORT INFORMATION
Selected seeds: All seeds
Export id: 9670f3399f774789b7c3e18975d25611
Export type: twitter_filter
Format: Full JSON
Export completed:  Oct. 30, 2017, 11:21:04 p.m. EDT
Deduplicate: No

COLLECTION INFORMATION
Collection name: test set for world series
Collection id: 34e3f7460b5c4df09d64a1e61fd81238
Collection set: mlb-test (collection set id d6e8c27b1bc942e78790aa55a82b3a7a)
Harvest type: Twitter filter
Collection description: running for just one hour, just for fun.

Harvest options:
Media: No
Web resources: No

Seeds:
* Track: dodgers,astros - Active

Change log:

Change to test set for world series (collection) on Oct. 30, 2017, 10:59:56 p.m. EDT by dchud:
* is_active: "True" changed to "False"

Change to test set for world series (collection) on Oct. 30, 2017, 10:58:51 p.m. EDT by dchud:
* is_on: "True" changed to "False"

Change to test set for world series (collection) on Oct. 29, 2017, 8:01:24 p.m. EDT by dch

The most important pieces in that metadata are:

 * It tracked tweets that mentioned "dodgers" or "astros".  Every item in this set should refer to one or the other, or both.
 * This data was not deduplicated; we may see individual items more than once.
 * Data was collected between October 29 and October 30.  Game 5 of the Series was played during this time.
 
You should not need to know anything about baseball to complete this assignment.

**Please note**: sometimes social media data contains offensive material.  This data set has not been filtered; if you do come across something inappropriate, please do your best to ignore it if you can.

## Fetch the data

The following files are available:

 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_005.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json
 
### Q1.1 - Upload the above files to your instance using `wget`.  Verify the file sizes using the command line. 

Each file should contain exactly 100,000 tweets.  

*Note*: you are required to use all files.  It will be easier to process more data if you use a larger EC2 instance type, as suggested above.  Use the exact same set of files throughout the assignment.

**Answer**

In [ ]:
# Upload the files using wget. Add more if needed.

In [4]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json

--2022-11-18 06:33:12--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.107.238, 52.216.112.150, 52.216.144.61, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.107.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 595711407 (568M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_003.json’

9670f3399f774789b7c 100%[===================>] 568.11M  77.5MB/s    in 6.9s    

2022-11-18 06:33:19 (82.1 MB/s) - ‘9670f3399f774789b7c3e18975d25611_003.json’ saved [595711407/595711407]



In [5]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json

--2022-11-18 06:33:24--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.112.150, 52.216.129.173, 52.216.244.254, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.112.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549995846 (525M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_004.json’

9670f3399f774789b7c 100%[===================>] 524.52M  84.2MB/s    in 6.6s    

2022-11-18 06:33:31 (79.2 MB/s) - ‘9670f3399f774789b7c3e18975d25611_004.json’ saved [549995846/549995846]



In [6]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_005.json

--2022-11-18 06:33:33--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_005.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.216.240, 52.217.78.62, 52.217.85.142, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.216.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 530698683 (506M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_005.json’

9670f3399f774789b7c 100%[===================>] 506.11M  83.8MB/s    in 6.3s    

2022-11-18 06:33:40 (80.8 MB/s) - ‘9670f3399f774789b7c3e18975d25611_005.json’ saved [530698683/530698683]



In [7]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json

--2022-11-18 06:33:42--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.53.160, 52.216.139.85, 52.216.184.213, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.53.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 545081593 (520M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_006.json’

9670f3399f774789b7c 100%[===================>] 519.83M  91.5MB/s    in 5.8s    

2022-11-18 06:33:48 (90.1 MB/s) - ‘9670f3399f774789b7c3e18975d25611_006.json’ saved [545081593/545081593]



#### Check the files have exactly 100,000 tweets using the command line

In [8]:
# Write you code here. 
!wc -l *.json

    100000 9670f3399f774789b7c3e18975d25611_003.json
    100000 9670f3399f774789b7c3e18975d25611_004.json
    100000 9670f3399f774789b7c3e18975d25611_005.json
    100000 9670f3399f774789b7c3e18975d25611_006.json
    400000 total


For your reference, here is the text of one Tweet, randomly selected from one of these files.  You might wish to study its structure and refer to it later.

In [9]:
!cat *.json | shuf -n 1 > example-tweet.json

In [10]:
import json
print(json.dumps(json.load(open("example-tweet.json")), indent=2))

{
  "quote_count": 0,
  "contributors": null,
  "truncated": false,
  "text": "\ud83d\udc40\ud83d\udc40\ud83d\udc40 https://t.co/wbd9uiFLnc",
  "is_quote_status": true,
  "in_reply_to_status_id": null,
  "reply_count": 0,
  "id": 924878222365425664,
  "favorite_count": 0,
  "entities": {
    "user_mentions": [],
    "symbols": [],
    "hashtags": [],
    "urls": [
      {
        "url": "https://t.co/wbd9uiFLnc",
        "indices": [
          4,
          27
        ],
        "expanded_url": "https://twitter.com/NikoNikosGreek/status/924840978598518785",
        "display_url": "twitter.com/NikoNikosGreek\u2026"
      }
    ]
  },
  "quoted_status_id": 924840978598518785,
  "retweeted": false,
  "coordinates": null,
  "timestamp_ms": "1509343126275",
  "quoted_status": {
    "quote_count": 13,
    "contributors": null,
    "truncated": true,
    "text": "If the Astros win and pull this off tonight I will sell my original lamb and beef gyros sandwiches tomorow at 50% o\u2026 https://t.

You can find several key elements in this example; the text, time, and language of the tweet, whether it was a reply to another user, the user's screen name along with their primary language and other account information like creation date, follower/friend/tweet counts, and perhaps their location.  
If there are hashtags, user mentions, or urls present in their tweet, they will be present in the `entities` section; these are not present in every tweet.  If this is a retweet, you will see the original tweet and its information nested within.

### Q1.2 - Start up Spark, and verify the file sizes.

We will use our normal startup sequence here:

In [9]:
import os

In [10]:
os.environ['SPARK_HOME'] = '/usr/local/lib/spark'

In [11]:
import findspark

In [12]:
findspark.init()

In [13]:
from pyspark import SparkContext

In [14]:
spark = SparkContext(appName='project-03')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/18 09:01:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
spark

<SparkContext master=local[*] appName=project-03>

In [16]:
from pyspark import SQLContext

In [17]:
sqlc = SQLContext(spark)

/usr/local/lib/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [18]:
sqlc

In [19]:
tweets = sqlc.read.json("*.json")

22/11/18 09:01:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Verify that Spark has loaded the same number of tweets you saw before:

**Answer**

In [ ]:
# EDIT THIS CELL

In [21]:
tweets.count()

400001

Do you see exactly the same number of tweets in Spark that you saw on the command line? 

In [24]:
tweets1 = sqlc.read.json("9670f3399f774789b7c3e18975d25611_003.json")

In [25]:
tweets1.count()

100000

In [26]:
tweets2 = sqlc.read.json("9670f3399f774789b7c3e18975d25611_004.json")

In [27]:
tweets2.count()

100000

In [28]:
tweets3 = sqlc.read.json("9670f3399f774789b7c3e18975d25611_005.json")

In [29]:
tweets3.count()

100000

In [30]:
tweets4 = sqlc.read.json("9670f3399f774789b7c3e18975d25611_006.json")

In [31]:
tweets4.count()

100000

## Part 2 - Comparing DataFrames and Spark SQL

For the next three questions, we will look at operations using both DataFrames and SQL queries. Note that `tweets` is already a DataFrame:

To issue SQL queries, we need to register a table based on `tweets`:

This is all well and good, but how well did schema inference work?

In [22]:
tweets.createOrReplaceTempView("tweets")

### Q2.1 - Which 10 languages are most commonly used in tweets?  Verify your result by executing it with both the dataframe and with SQL.

Hint: for the dataframe, use `groupBy`, `count`, and `orderBy`.  See the documentation at https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html for details on these and other functions.

**Answer**

In [ ]:
# Edit this cell and write query using dataframe

In [37]:
tweets.columns

['contributors',
 'coordinates',
 'created_at',
 'display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'reply_count',
 'retweet_count',
 'retweeted',
 'retweeted_status',
 'source',
 'text',
 'timestamp_ms',
 'truncated',
 'user']

In [38]:
tweets.groupby('lang').count().orderBy('count', ascending=False).show(10)

+----+------+
|lang| count|
+----+------+
|  en|346302|
|  es| 31869|
| und| 14660|
|  in|  2355|
|  fr|   997|
|  pt|   914|
|  tl|   485|
|  ht|   377|
|  nl|   347|
|  it|   285|
+----+------+
only showing top 10 rows



In [ ]:
# Edit this cell and write query using SQL

In [39]:
sqlc.sql("""
    SELECT lang,count(*)
    FROM tweets
    Group by lang
    Order by count(*) DESC
""").show(10)

+----+--------+
|lang|count(1)|
+----+--------+
|  en|  346302|
|  es|   31869|
| und|   14660|
|  in|    2355|
|  fr|     997|
|  pt|     914|
|  tl|     485|
|  ht|     377|
|  nl|     347|
|  it|     285|
+----+--------+
only showing top 10 rows



Write your observations here.
The results are the same.

### Q2.2 - Which 10 time zones are most common among users?  Verify your result with both the dataframe and SQL.

*Note*: for this question, you may leave NULL values present in your results, as a way to help you understand what data is present and what is missing.

**Answer**

In [ ]:
# Edit this cell and write query using dataframe

In [40]:
tweets.groupby('user.time_zone').count().orderBy('count', ascending=False).show(10)

+--------------------+------+
|           time_zone| count|
+--------------------+------+
|                null|166386|
|Pacific Time (US ...| 72214|
|Central Time (US ...| 63665|
|Eastern Time (US ...| 35889|
|             Arizona| 10748|
|Mountain Time (US...| 10057|
|Atlantic Time (Ca...|  4805|
|               Quito|  4397|
|             Caracas|  4218|
|         Mexico City|  3935|
+--------------------+------+
only showing top 10 rows



In [ ]:
# Edit this cell and write query using SQL

In [41]:
sqlc.sql("""
    SELECT user.time_zone,count(*)
    FROM tweets
    where user.time_zone is not null
    Group by user.time_zone
    Order by count(*) DESC
""").show(10)

+--------------------+--------+
|           time_zone|count(1)|
+--------------------+--------+
|Pacific Time (US ...|   72214|
|Central Time (US ...|   63665|
|Eastern Time (US ...|   35889|
|             Arizona|   10748|
|Mountain Time (US...|   10057|
|Atlantic Time (Ca...|    4805|
|               Quito|    4397|
|             Caracas|    4218|
|         Mexico City|    3935|
|              Hawaii|    3489|
+--------------------+--------+
only showing top 10 rows



Write your observations here.
The results are the same.

### Q2.3 - How many tweets mention the Dodgers?  How many mention the Astros?  How many mention both?

You may use either the dataframe or SQL to answer.  Explain why you have chosen that approach.

Hint:  you will want to look at the value of the `text` field.

**Answer**

In [ ]:
# Edit this cell and add more as appropriate

In [42]:
sqlc.sql("""
    SELECT text
    FROM tweets
    where text ilike '%Dodgers%'
""").count()

168218

In [43]:
sqlc.sql("""
    SELECT text
    FROM tweets
    where text ilike '%Astros%'
""").count()

246465

In [44]:
sqlc.sql("""
    SELECT text
    FROM tweets
    where text ilike '%Astros%' 
    and text ilike '%Dodgers%'
""").count()

49545

## Part 3 - More complex queries

For this section, you may choose to use dataframe queries or SQL.  If you wish, you may verify results by using both, as in Part 2, but this is not required for this section.

### Q3.1 - Team mentions by location

In which users' locations are the Astros and the Dodgers being mentioned the most?  Consider each team separately, one at a time.  Discuss your findings. Do not count null time_zones or location.

Hint:  you may use either the time zones or user-specified locations for this question.


**Answer**

In [ ]:
# Edit this cell and add more as appropriate

In [23]:
sqlc.sql("""
    SELECT user.time_zone,user.location,count(*)
    FROM tweets
    where text ilike '%Astros%' 
    and user.location is not null
    and user.time_zone is not null
    Group by user.time_zone,user.location
    Order by count(*) DESC
""").show(5)

+--------------------+-----------+--------+
|           time_zone|   location|count(1)|
+--------------------+-----------+--------+
|Central Time (US ...|Houston, TX|    5742|
|Pacific Time (US ...|Houston, TX|    1529|
|Central Time (US ...| Texas, USA|    1288|
|Central Time (US ...|      Texas|    1232|
|Eastern Time (US ...|Houston, TX|    1223|
+--------------------+-----------+--------+
only showing top 5 rows



In [24]:
sqlc.sql("""
    SELECT user.time_zone,user.location,count(*)
    FROM tweets
    where text ilike '%Dodgers%' 
    and user.location is not null
    and user.time_zone is not null
    Group by user.time_zone,user.location
    Order by count(*) DESC
""").show(5)

+--------------------+---------------+--------+
|           time_zone|       location|count(1)|
+--------------------+---------------+--------+
|Pacific Time (US ...|Los Angeles, CA|    2972|
|Pacific Time (US ...|    Los Angeles|    1280|
|Pacific Time (US ...|California, USA|     873|
|Central Time (US ...|    Houston, TX|     796|
|Pacific Time (US ...|     California|     503|
+--------------------+---------------+--------+
only showing top 5 rows



Write your observations here.
The Astros is mentioned the most in Houston, TX. The Dodgers is mentioned most in Los Angeles, CA.

### Q3.2 - Which Twitter users are being replied to the most?

Discuss your findings.

Hint: use the top-level `in_reply_to_screen_name` for this.

**Answer**

In [ ]:
# Edit this cell and add more as appropriate

In [25]:
sqlc.sql("""
    SELECT in_reply_to_screen_name,count(*)
    FROM tweets
    Where in_reply_to_screen_name is not null
    group by in_reply_to_screen_name
    order by count(*) DESC    
""").show(2)

+-----------------------+--------+
|in_reply_to_screen_name|count(1)|
+-----------------------+--------+
|                 astros|    4034|
|                Dodgers|    3297|
+-----------------------+--------+
only showing top 2 rows



Write your observations here
The astros is being replied to the most.

### Q3.3 - Which 10 verified users have the most followers?  Which 10 unverified users have the most followers?

Provide both the screen names (screen_name) and follower counts (followers_count) for each.
Verified users -- use verified == 't'

Discuss your findings.

**Answer**

In [ ]:
# Edit this cell and add more as appropriate

In [26]:
sqlc.sql("""
    SELECT user.id,user.screen_name,user.followers_count
    FROM tweets
    WHERE user.screen_name is not null
    and user.verified != 't'
    group by user.id,user.screen_name,user.followers_count
    order by user.followers_count DESC
""").show(10)

+---------+---------------+---------------+
|       id|    screen_name|followers_count|
+---------+---------------+---------------+
| 99943864|Daminous_Purity|         998742|
| 99943864|Daminous_Purity|         998731|
| 99943864|Daminous_Purity|         998730|
| 99943864|Daminous_Purity|         998722|
| 99943864|Daminous_Purity|         998720|
| 99943864|Daminous_Purity|         998684|
| 29614331|        chochos|         833669|
|202409095|     TexasHumor|         816301|
|202409095|     TexasHumor|         816295|
| 82971772|  el_carabobeno|         725952|
+---------+---------------+---------------+
only showing top 10 rows



In [27]:
sqlc.sql("""
    SELECT user.id,user.screen_name,user.followers_count,user.verified
    FROM tweets
    WHERE user.screen_name is not null
    and user.verified == 't'
    order by user.followers_count DESC
""").show(10)

+---------+--------------+---------------+--------+
|       id|   screen_name|followers_count|verified|
+---------+--------------+---------------+--------+
|   428333|        cnnbrk|       53191119|    true|
|   807095|       nytimes|       39959480|    true|
|   759251|           CNN|       38209973|    true|
|  1652541|       Reuters|       18937529|    true|
|  1367531|       FoxNews|       16272836|    true|
| 28785486|           ABC|       12551437|    true|
|  2467791|washingtonpost|       11417638|    true|
|236636515|   lopezdoriga|        7859821|    true|
| 18479513|           MLB|        7841255|    true|
| 18479513|           MLB|        7840827|    true|
+---------+--------------+---------------+--------+
only showing top 10 rows



Write your observations here.


In [ ]:
Verified users: Daminous_Purity,chochos,TexasHumor,el_carabobeno have the most followers. 
Unverified users: cnnbrk,nytimes,CNN,Reuters,FoxNews,ABC,washingtonpost,lopezdoriga,MLB


### Q4 - Analyze common words in tweet text

Following the example in class, use `tweets.rdd` to find the most common interesting words in tweet text.  To keep it "interesting", remove at least 10 common stop words found in tweets, like "a", "an", "the", and "RT" (you might want to derive these stop words from initial results).  A simple split on text whitespace like we had in class is sufficient; you do not have to account for punctuation. 

After you find the most common words, use dataframe or SQL queries to find patterns among how those words are used.  For example, are they more frequently used by Dodgers or Astros fans, or by people in one part of the country over another?  Explore and see what you can find, and discuss your findings.

You will notice that common words include words like "thisteam" and "earnhistory". I would like you to write two queries to investigate whether those two words are used by the Astros or Dodgers

Hint: don't forget all the word count pipeline steps we used earlier in class.

**Answer**

In [ ]:
# Edit this cell and add more as appropriate

In [29]:
stopword = ['a','an','RT','to','the','in','am','is','are','I','and','of',]

In [30]:
word_count =tweets.rdd.flatMap(lambda r: r['text'].split(' ')) \
    .filter(lambda x : x not in stopword) \
    .map(lambda t: (t, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(10, key=lambda pair: -pair[1]) 

In [31]:
word_count

[('Astros', 69114),
 ('', 62332),
 ('Dodgers', 52408),
 ('@astros:', 49821),
 ('#WorldSeries', 49706),
 ('for', 38574),
 ('@astros', 37272),
 ('#ThisTeam', 36593),
 ('#EarnHistory', 34614),
 ('this', 33225)]

In [32]:
# check the time_zone and location of the Astros
sqlc.sql("""
    SELECT text, user.time_zone,user.location ,count(*)
    FROM tweets
    where text ilike '%Astros%' 
    and user.time_zone is not null
    and user.location is not null
    group by text,user.time_zone,user.location
    order by count(*) desc
    """).show(5)

+--------------------+--------------------+-----------+--------+
|                text|           time_zone|   location|count(1)|
+--------------------+--------------------+-----------+--------+
|RT @astros: The k...|Central Time (US ...|Houston, TX|     175|
|RT @astros: Corre...|Central Time (US ...|Houston, TX|     151|
|RT @astros: BREGM...|Central Time (US ...|Houston, TX|     150|
|RT @astros: 🐐🐐?...|Central Time (US ...|Houston, TX|     127|
|RT @astros: H-Tow...|Central Time (US ...|Houston, TX|      82|
+--------------------+--------------------+-----------+--------+
only showing top 5 rows



In [33]:
sqlc.sql("""
    SELECT user.location ,count(*)
    FROM tweets
    where text ilike '%Astros%' 
    and user.time_zone is not null
    and user.location is not null
    group by user.location
    order by count(*) desc
    """).show(10)

+---------------+--------+
|       location|count(1)|
+---------------+--------+
|    Houston, TX|    9584|
|     Texas, USA|    1991|
|          Texas|    1797|
|Los Angeles, CA|    1639|
|        Houston|    1636|
| Houston, Texas|    1485|
|     Austin, TX|    1047|
|  United States|     882|
|    Houston, Tx|     661|
|San Antonio, TX|     619|
+---------------+--------+
only showing top 10 rows



In [34]:
sqlc.sql("""
    SELECT user.location ,count(*)
    FROM tweets
    where text like '%Dodgers%' 
    and user.time_zone is not null
    and user.location is not null
    group by user.location
    order by count(*) desc
    """).show(10)

+-------------------+--------+
|           location|count(1)|
+-------------------+--------+
|    Los Angeles, CA|    3452|
|        Los Angeles|    1429|
|        Houston, TX|    1131|
|    California, USA|     949|
|      United States|     561|
|          Venezuela|     543|
|             México|     539|
|         California|     519|
|         Texas, USA|     410|
|Southern California|     328|
+-------------------+--------+
only showing top 10 rows



In [35]:
# Find dodgers offical account id and name
sqlc.sql("""
    SELECT user.id,user.name, user.followers_count
    FROM tweets
    where user.name ilike '%Dodgers%'  
    order by user.followers_count desc
    """).show()

+----------+-------------------+---------------+
|        id|               name|followers_count|
+----------+-------------------+---------------+
|  23043294|Los Angeles Dodgers|        1830535|
|  23043294|Los Angeles Dodgers|        1829283|
|  23043294|Los Angeles Dodgers|        1828991|
|  23043294|Los Angeles Dodgers|        1828850|
|  23043294|Los Angeles Dodgers|        1828646|
|  23043294|Los Angeles Dodgers|        1828364|
|  23043294|Los Angeles Dodgers|        1828222|
|  23043294|Los Angeles Dodgers|        1828208|
|  23043294|Los Angeles Dodgers|        1827973|
|  23043294|Los Angeles Dodgers|        1827147|
|  23043294|Los Angeles Dodgers|        1826697|
|  23043294|Los Angeles Dodgers|        1826582|
|  23043294|Los Angeles Dodgers|        1826491|
|  23043294|Los Angeles Dodgers|        1826394|
|  23043294|Los Angeles Dodgers|        1826010|
|  23043294|Los Angeles Dodgers|        1825865|
|2226855828| EL PINCHE DODGERS™|         144402|
|2226855828| EL PINC

In [36]:
# Find Astros offical accounr id and name 
sqlc.sql("""
    SELECT user.id,user.name, user.followers_count
    FROM tweets
    where user.name ilike '%Astros%'  
    order by user.followers_count desc
    """).show()

+--------+--------------+---------------+
|      id|          name|followers_count|
+--------+--------------+---------------+
|52803520|Houston Astros|        1086022|
|52803520|Houston Astros|        1085833|
|52803520|Houston Astros|        1085644|
|52803520|Houston Astros|        1085349|
|52803520|Houston Astros|        1085287|
|52803520|Houston Astros|        1085125|
|52803520|Houston Astros|        1084874|
|52803520|Houston Astros|        1084132|
|52803520|Houston Astros|        1083839|
|52803520|Houston Astros|        1083651|
|52803520|Houston Astros|        1083555|
|52803520|Houston Astros|        1083397|
|52803520|Houston Astros|        1083048|
|52803520|Houston Astros|        1082771|
|52803520|Houston Astros|        1082678|
|52803520|Houston Astros|        1082221|
|52803520|Houston Astros|        1082085|
|52803520|Houston Astros|        1081998|
|52803520|Houston Astros|        1081898|
|52803520|Houston Astros|        1080761|
+--------+--------------+---------

In [37]:
# Find whether Astros use the word thisteam and earnhistory
sqlc.sql("""
    SELECT text, user.id, user.name
    FROM tweets
    where user.id == '52803520' 
    and text ilike "%ThisTeam%"
    and text ilike "%EarnHistory%"
    """).count()

0

In [38]:
sqlc.sql("""
    SELECT text
    FROM tweets
    where user.id like '52803520' 
    and text ilike "%ThisTeam%"
    and text ilike "%EarnHistory%"
    """).show()

+----+
|text|
+----+
+----+



In [39]:
# find whether Astros use the word thisteam and earn history
sqlc.sql("""
    SELECT text
    FROM tweets
    where user.id like '23043294' 
    and text ilike "%ThisTeam%"
    and text ilike "%EarnHistory%"
    """).count()

0